In [40]:
from pathlib import Path

# pip install sentence-transformers
# pip install tf-keras

In [41]:
def clean_text(text):
    text = text.replace("\n", " ")
    text = " ".join(text.split())
    return text


def chunk_text(text, chunk_size=250, overlap=50):
    words = text.split()
    chunks = []

    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(" ".join(chunk))
        start = end - overlap

    return chunks


def load_documents(folder_path):
    documents = []
    for file in Path(folder_path).glob("*.txt"):
        with open(file, "r", encoding="utf-8") as f:
            documents.append(f.read())
    return documents

In [42]:
# Step 1 - Prepare your documents

documents = load_documents("docs")

all_chunks = []

for doc in documents:
    cleaned_doc = clean_text(doc)
    chunks = chunk_text(cleaned_doc)
    all_chunks.extend(chunks)

In [43]:
print(all_chunks)
print(len(all_chunks))

['Mohammed Ali Jinnah (born December 25, 1876?, Karachi, India [now in Pakistan]—died September 11, 1948, Karachi) was an Indian Muslim politician, who was the founder and first governor-general (1947–48) of Pakistan. Early years Jinnah was the eldest of seven children of Jinnahbhai Poonja, a prosperous merchant, and his wife, Mithibai. His family was a member of the Khoja caste, Hindus who had converted to Islam centuries earlier and who were followers of the Aga Khan. There is some question about Jinnah’s date of birth: although he maintained that it was December 25, 1876, school records from Karachi (Pakistan) give a date of October 20, 1875. After being taught at home, Jinnah was sent in 1887 to the Sind Madrasat al-Islam (now Sindh Madressatul Islam University) in Karachi. Later he attended the Christian Missionary Society High School (also in Karachi), where at the age of 16 he passed the matriculation examination of the University of Bombay (now University of Mumbai, in Mumbai, 

In [44]:
# Step 2 - Create embeddings

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [45]:
# Create Embeddings for All Chunks

embeddings = model.encode(all_chunks, convert_to_numpy=True)

In [46]:
embeddings[0]

array([-4.44593877e-02,  1.08115800e-01, -7.15348274e-02,  2.20579095e-02,
        2.76173488e-03, -1.36383157e-02,  7.81506393e-03, -1.98730584e-02,
       -1.12462193e-02,  5.52694649e-02,  9.06524509e-02, -2.36261617e-02,
        9.87411961e-02,  3.43778287e-03,  6.71548396e-02,  7.62551371e-03,
       -5.06169833e-02,  3.81875187e-02, -8.44949763e-03, -7.81465918e-02,
       -3.98823619e-02,  3.72276194e-02, -1.39758061e-03, -3.99890468e-02,
       -3.25423442e-02,  1.52916592e-02,  6.36922866e-02, -9.40147415e-02,
       -5.47262691e-02,  7.24705607e-02, -4.59465906e-02, -4.31373268e-02,
       -3.27388197e-02,  1.31238597e-02, -3.59168537e-02, -2.76860725e-02,
        1.00576337e-02,  1.13265701e-01,  1.56634077e-02, -2.59070415e-02,
        5.80268838e-02,  1.91894695e-02, -3.24234553e-02, -2.32311767e-02,
        8.49992782e-02, -3.55843082e-02,  3.35635021e-02, -2.33238544e-02,
       -5.85673004e-03, -2.96404492e-02, -7.76314661e-02,  3.17060277e-02,
       -1.31645743e-02, -

In [47]:
print(embeddings.shape)

# (number_of_chunks, 384)

(13, 384)


In [48]:
# Save Embeddings

import numpy as np

np.save("chunk_embeddings.npy", embeddings)

### Step 3 - Build a simple similarity search


In [49]:
# From Step 1 & 2, you already have:

print(all_chunks)  # List[str]
print(embeddings)  # numpy.ndarray, shape = (num_chunks, 384)
print(model)  # SentenceTransformer("all-MiniLM-L6-v2")

['Mohammed Ali Jinnah (born December 25, 1876?, Karachi, India [now in Pakistan]—died September 11, 1948, Karachi) was an Indian Muslim politician, who was the founder and first governor-general (1947–48) of Pakistan. Early years Jinnah was the eldest of seven children of Jinnahbhai Poonja, a prosperous merchant, and his wife, Mithibai. His family was a member of the Khoja caste, Hindus who had converted to Islam centuries earlier and who were followers of the Aga Khan. There is some question about Jinnah’s date of birth: although he maintained that it was December 25, 1876, school records from Karachi (Pakistan) give a date of October 20, 1875. After being taught at home, Jinnah was sent in 1887 to the Sind Madrasat al-Islam (now Sindh Madressatul Islam University) in Karachi. Later he attended the Christian Missionary Society High School (also in Karachi), where at the age of 16 he passed the matriculation examination of the University of Bombay (now University of Mumbai, in Mumbai, 

In [50]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Convert User Question into Embedding
def embed_question(question, model):
    return model.encode([question], convert_to_numpy=True)


# Compute Similarity Scores
def compute_similarity(question_embedding, chunk_embeddings):
    similarities = cosine_similarity(question_embedding, chunk_embeddings)
    return similarities[0]  # flatten array


# Retrieve Top 3 Relevant Chunks
def retrieve_top_chunks(question, chunks, embeddings, model, top_k=3):
    question_embedding = embed_question(question, model)
    similarity_scores = compute_similarity(question_embedding, embeddings)

    top_indices = np.argsort(similarity_scores)[-top_k:][::-1]

    top_chunks = [chunks[i] for i in top_indices]
    return top_chunks

In [65]:
question = "What is name of Quaid-e-azam's father?"

top_chunks = retrieve_top_chunks(
    question=question, chunks=all_chunks, embeddings=embeddings, model=model
)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")


--- Chunk 1 ---
Mohammed Ali Jinnah (born December 25, 1876?, Karachi, India [now in Pakistan]—died September 11, 1948, Karachi) was an Indian Muslim politician, who was the founder and first governor-general (1947–48) of Pakistan. Early years Jinnah was the eldest of seven children of Jinnahbhai Poonja, a prosperous merchant, and his wife, Mithibai. His family was a member of the Khoja caste, Hindus who had converted to Islam centuries earlier and who were followers of the Aga Khan. There is some question about Jinnah’s date of birth: although he maintained that it was December 25, 1876, school records from Karachi (Pakistan) give a date of October 20, 1875. After being taught at home, Jinnah was sent in 1887 to the Sind Madrasat al-Islam (now Sindh Madressatul Islam University) in Karachi. Later he attended the Christian Missionary Society High School (also in Karachi), where at the age of 16 he passed the matriculation examination of the University of Bombay (now University of Mumb

### Step 4 - Build the RAG prompt


In [66]:
def build_rag_prompt(question, context_chunks):
    context = "\n\n".join(context_chunks)

    prompt = f"""
    Use only the following context to answer the question:

    {context}

    Question: {question}
    Answer:
    """

    return prompt

In [67]:
question = "What is name of Quaid-e-azam's father?"


In [68]:
rag_prompt = build_rag_prompt(question, top_chunks)

print(rag_prompt)



    Use only the following context to answer the question:

    Mohammed Ali Jinnah (born December 25, 1876?, Karachi, India [now in Pakistan]—died September 11, 1948, Karachi) was an Indian Muslim politician, who was the founder and first governor-general (1947–48) of Pakistan. Early years Jinnah was the eldest of seven children of Jinnahbhai Poonja, a prosperous merchant, and his wife, Mithibai. His family was a member of the Khoja caste, Hindus who had converted to Islam centuries earlier and who were followers of the Aga Khan. There is some question about Jinnah’s date of birth: although he maintained that it was December 25, 1876, school records from Karachi (Pakistan) give a date of October 20, 1875. After being taught at home, Jinnah was sent in 1887 to the Sind Madrasat al-Islam (now Sindh Madressatul Islam University) in Karachi. Later he attended the Christian Missionary Society High School (also in Karachi), where at the age of 16 he passed the matriculation examination of 

In [69]:
import ollama

response = ollama.chat(
    model="llama3.2",
    messages=[{"role": "user", "content": rag_prompt}]
)

print(response["message"]["content"])


Jinnahbhai Poonja.


### Step 5 - Test RAG vs No-RAG

In [62]:
questions = [
    "What is an RNN?",
    "Why do RNNs suffer from vanishing gradients?",
    "What is LLaMA?",
    "How does a transformer differ from an RNN?",
    "What is Retrieval-Augmented Generation?",
    "Why does RAG reduce hallucinations?",
    "What role do embeddings play in RAG?",
    "What are common applications of RAG?",
    "Why are transformers better at long-range dependencies?",
    "How does similarity search work in RAG?"
]


In [63]:
# LLaMA Without RAG
def ask_llama_no_rag(question):
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": question}]
    )
    return response["message"]["content"]

In [64]:
def ask_llama_with_rag(question, chunks, embeddings, model):
    top_chunks = retrieve_top_chunks(
        question=question,
        chunks=chunks,
        embeddings=embeddings,
        model=model
    )

    prompt = build_rag_prompt(question, top_chunks)

    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"]
